In [39]:
import fitz  # PyMuPDF
import pandas as pd

pdf_file = r"C:\Users\alsot\Downloads\CV_TAUA_SANTOS_ENG.pdf" 


def extract_paragraphs(pdf_path):
    doc = fitz.open(pdf_path)  # Open the PDF
    paragraphs = []  # List to store paragraphs
    
    # Loop through pages in the PDF
    for page in doc:
        text = page.get_text("text")  # Extract text as plain text
        # Split text by blank lines to identify paragraphs
        paragraphs_in_page = text.split("\n\n")  # Two newline characters as separator
        paragraphs.extend(paragraphs_in_page)  # Add to the list

    return paragraphs

def save_paragraphs_to_dataframe(paragraphs):
    # Create a DataFrame from the paragraphs
    df = pd.DataFrame(paragraphs, columns=["Paragraph"])
    return df

# Example usage

paragraphs = extract_paragraphs(pdf_file)
df_paragraphs = save_paragraphs_to_dataframe(paragraphs)

# Show the DataFrame (first few rows)
print(df_paragraphs.head())

# Optionally, you can save it to a CSV file
df_paragraphs.to_csv("paragraphs.csv", index=False)


                                           Paragraph
0   \n \nTAUÃ SANTOS \nVila Mariana , São Paulo, ...
1   \n \n  \n \n \n \n \n \n  \n \n \nworkflows, ...


In [82]:
import PyPDF2
import pandas as pd

# Path to your PDF file
pdf_file = r"C:\Users\alsot\Downloads\CV_TAUA_SANTOS_ENG.pdf" 

# Open the PDF file
with open(pdf_file, "rb") as file:
    reader = PyPDF2.PdfReader(file)
    
    # Extract text from each page
    text = ""
    for page in reader.pages:
        text += page.extract_text()

# Reconstruct paragraphs
paragraphs = []
current_paragraph = ""

# Split the text into lines
lines = text.split("\n")

blank_count = 0

# Iterate through the lines and reconstruct paragraphs
for line in lines:
    if line == ' ':
        blank_count+=1
    else:
        blank_count = 0

    if blank_count < 2:
        current_paragraph += line
    else:
        paragraphs.append(current_paragraph)
        current_paragraph = ''
    
    
    # if line.strip() == "":  # Empty line indicates a paragraph break
    #     if current_paragraph.strip() != "":  # Avoid adding empty paragraphs
    #         paragraphs.append(current_paragraph.strip())
    #         current_paragraph = ""
    # else:
    #     current_paragraph += line + " "  # Add the line to the current paragraph

# Add the last paragraph (if any)
if current_paragraph.strip() != "":
    paragraphs.append(current_paragraph.strip())

# Create a pandas DataFrame
df = pd.DataFrame(paragraphs, columns=["Paragraph"])


In [93]:
df['Paragraph'].dropna()

0                                                      
1     TAUÃ SANTOS Vila Mariana , São Paulo, SP | tau...
2                                                      
3     EDUCATION SÃO PAULO UNIVERSITY (POLYTECHNIC SC...
4     Junior Product Analyst | 04/2021 - 04/2022 ● P...
5                                                      
6                                                      
7                                                      
8                                                      
9                                                      
10                                                     
11                                                     
12                                                     
13                                                     
14                                                     
15    workflows, and supported compliance with inter...
16                                                     
17    SUPPLEMENTARY EDUCATION Business Training 